# Скачиваем данные и устанавливаем программы

In [ ]:
!git clone https://github.com/nevermarine/hse22_hw2/ 
!cd hse22_hw2 && mv gms2.lst proteins.fasta scaffolds.hits_from_MIL_1.txt scaffolds.hits_from_SwissProt.txt /content
!rm hse22_hw2 -rf

Cloning into 'hse22_hw2'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [ ]:
!yes | sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 4.7 MB/s 


## Аннотация генома 


Biopython Tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html

In [ ]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [ ]:
sc = {}
for n in SeqIO.parse("out_gapClosed.fa", "fasta"):
  n.annotations['molecule_type'] = 'DNA'
  n.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  n.annotations['data_file_division'] = 'BCT'
  
  sc[n.id] = n

In [ ]:
genes = {}
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaffold = desc[1]
  start, end = int(desc[2]), int(desc[3])
  strand = 1 if desc[4] == '+' else -1
  
  feature = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feature.qualifiers['locus_tag'] = [desc[0]]
  feature.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feature)
  
  genes[desc[0]] = feature

In [ ]:
feature.qualifiers

OrderedDict([('locus_tag', ['3622']),
             ('translation', [Seq('PVFVPAASVDEHDFFHSVLLHALDLYGFFDSHKR')])])

## Добавляем функции белков (из бактерии MIL-1)

In [ ]:
mil1genes = dict()
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in feat.qualifiers:
    continue
  if 'product' not in feat.qualifiers:
    continue
  
  mil1genes[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

In [ ]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil1hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10515,3620,lcl|HF680312.1_prot_1833,100.000,52,0,0,1,52,95,146,5.410000e-34,109.0
10516,3620,lcl|HF680312.1_prot_CCU71654.1_1205,100.000,52,0,0,1,52,95,146,2.250000e-33,110.0
10517,3620,lcl|HF680312.1_prot_CCU71593.1_1144,100.000,52,0,0,1,52,95,146,2.510000e-33,110.0
10518,3620,lcl|HF680312.1_prot_CCU71934.1_1485,100.000,52,0,0,1,52,95,146,4.490000e-33,110.0


In [ ]:
hits = mil1hits[mil1hits['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2831,1024,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
4237,1485,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
9153,3221,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
2609,948,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
4001,1453,lcl|HF680312.1_prot_CCU70690.1_241,99.392,1645,10,0,1,1645,1,1645,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9919,3490,lcl|HF680312.1_prot_CCU71629.1_1180,61.224,49,8,1,2,39,139,187,1.280000e-11,54.7
4805,1682,lcl|HF680312.1_prot_CCU72581.1_2132,29.897,97,62,2,8,103,5,96,1.010000e-11,53.5
8066,2818,lcl|HF680312.1_prot_CCU72336.1_1887,29.897,97,62,2,8,103,5,96,1.010000e-11,53.5
10299,3584,lcl|HF680312.1_prot_CCU70460.1_11,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5


In [ ]:
for i, hit in hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil1genes[match]]

## Добавляем функции белков (из БД SwissProt)

Читаем информацию из файла scaffolds.hits_from_SwissProt.txt

In [ ]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

--2022-10-19 15:25:17--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  15.5MB/s    in 17s     

2022-10-19 15:25:33 (36.9 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [ ]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [ ]:
previd = None
swissgenes = dict()
for line in open('SwissProt_names.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

In [ ]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

In [ ]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

69